# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
%pip install -U scikit-learn

You should consider upgrading via the '/opt/homebrew/Cellar/jupyterlab/3.2.9/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
tf.__version__

'2.8.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(50)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [4]:
dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [5]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [6]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X[:, 2])

[0 0 0 ... 0 1 0]


In [7]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
np.shape(X_train)

(8000, 12)

In [11]:
X_train

array([[-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  0.32472465]])

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [12]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='softmax'))

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compiling the ANN

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Part 3 - Training the ANN

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 1000)

Epoch 1/1000
155/250 [=================>............] - ETA: 0s - loss: 0.6100 - accuracy: 0.7177 

2022-03-04 23:57:34.998381: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


250/250 [==============================] - 0s 316us/step - loss: 0.5735 - accuracy: 0.7460
Epoch 2/1000
250/250 [==============================] - 0s 300us/step - loss: 0.4765 - accuracy: 0.7960
Epoch 3/1000
250/250 [==============================] - 0s 504us/step - loss: 0.4431 - accuracy: 0.7961
Epoch 4/1000
250/250 [==============================] - 0s 297us/step - loss: 0.4224 - accuracy: 0.8159
Epoch 5/1000
250/250 [==============================] - 0s 293us/step - loss: 0.4101 - accuracy: 0.8195
Epoch 6/1000
250/250 [==============================] - 0s 294us/step - loss: 0.4020 - accuracy: 0.8220
Epoch 7/1000
250/250 [==============================] - 0s 296us/step - loss: 0.3935 - accuracy: 0.8254
Epoch 8/1000
250/250 [==============================] - 0s 294us/step - loss: 0.3843 - accuracy: 0.8386
Epoch 9/1000
250/250 [==============================] - 0s 295us/step - loss: 0.3742 - accuracy: 0.8465
Epoch 10/1000
250/250 [==============================] - 0s 294us/step - loss

250/250 [==============================] - 0s 298us/step - loss: 0.3349 - accuracy: 0.8634
Epoch 80/1000
250/250 [==============================] - 0s 295us/step - loss: 0.3342 - accuracy: 0.8624
Epoch 81/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3349 - accuracy: 0.8630
Epoch 82/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3347 - accuracy: 0.8630
Epoch 83/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3345 - accuracy: 0.8627
Epoch 84/1000
250/250 [==============================] - 0s 294us/step - loss: 0.3346 - accuracy: 0.8630
Epoch 85/1000
250/250 [==============================] - 0s 295us/step - loss: 0.3346 - accuracy: 0.8620
Epoch 86/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3343 - accuracy: 0.8631
Epoch 87/1000
250/250 [==============================] - 0s 293us/step - loss: 0.3347 - accuracy: 0.8621
Epoch 88/1000
250/250 [==============================] - 0s 301us/ste

250/250 [==============================] - 0s 296us/step - loss: 0.3333 - accuracy: 0.8639
Epoch 157/1000
250/250 [==============================] - 0s 299us/step - loss: 0.3334 - accuracy: 0.8634
Epoch 158/1000
250/250 [==============================] - 0s 304us/step - loss: 0.3333 - accuracy: 0.8630
Epoch 159/1000
250/250 [==============================] - 0s 305us/step - loss: 0.3334 - accuracy: 0.8631
Epoch 160/1000
250/250 [==============================] - 0s 300us/step - loss: 0.3330 - accuracy: 0.8644
Epoch 161/1000
250/250 [==============================] - 0s 295us/step - loss: 0.3331 - accuracy: 0.8648
Epoch 162/1000
250/250 [==============================] - 0s 295us/step - loss: 0.3333 - accuracy: 0.8633
Epoch 163/1000
250/250 [==============================] - 0s 298us/step - loss: 0.3331 - accuracy: 0.8637
Epoch 164/1000
250/250 [==============================] - 0s 303us/step - loss: 0.3332 - accuracy: 0.8631
Epoch 165/1000
250/250 [==============================] - 0s 

...

250/250 [==============================] - 0s 295us/step - loss: 0.3309 - accuracy: 0.8639
Epoch 619/1000
250/250 [==============================] - 0s 287us/step - loss: 0.3307 - accuracy: 0.8649
Epoch 620/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3309 - accuracy: 0.8656
Epoch 621/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3308 - accuracy: 0.8640
Epoch 622/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3310 - accuracy: 0.8641
Epoch 623/1000
250/250 [==============================] - 0s 290us/step - loss: 0.3308 - accuracy: 0.8646
Epoch 624/1000
250/250 [==============================] - 0s 290us/step - loss: 0.3310 - accuracy: 0.8631
Epoch 625/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3310 - accuracy: 0.8656
Epoch 626/1000
250/250 [==============================] - 0s 290us/step - loss: 0.3311 - accuracy: 0.8634
Epoch 627/1000
250/250 [==============================] - 0s 

250/250 [==============================] - 0s 296us/step - loss: 0.3311 - accuracy: 0.8650
Epoch 696/1000
250/250 [==============================] - 0s 293us/step - loss: 0.3310 - accuracy: 0.8640
Epoch 697/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3309 - accuracy: 0.8641
Epoch 698/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3309 - accuracy: 0.8639
Epoch 699/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3308 - accuracy: 0.8648
Epoch 700/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3308 - accuracy: 0.8651
Epoch 701/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3309 - accuracy: 0.8634
Epoch 702/1000
250/250 [==============================] - 0s 290us/step - loss: 0.3310 - accuracy: 0.8649
Epoch 703/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3309 - accuracy: 0.8649
Epoch 704/1000
250/250 [==============================] - 0s 

250/250 [==============================] - 0s 296us/step - loss: 0.3306 - accuracy: 0.8651
Epoch 773/1000
250/250 [==============================] - 0s 294us/step - loss: 0.3308 - accuracy: 0.8645
Epoch 774/1000
250/250 [==============================] - 0s 290us/step - loss: 0.3308 - accuracy: 0.8641
Epoch 775/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3309 - accuracy: 0.8643
Epoch 776/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3308 - accuracy: 0.8643
Epoch 777/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3309 - accuracy: 0.8659
Epoch 778/1000
250/250 [==============================] - 0s 290us/step - loss: 0.3308 - accuracy: 0.8644
Epoch 779/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3308 - accuracy: 0.8637
Epoch 780/1000
250/250 [==============================] - 0s 290us/step - loss: 0.3308 - accuracy: 0.8651
Epoch 781/1000
250/250 [==============================] - 0s 

250/250 [==============================] - 0s 298us/step - loss: 0.3310 - accuracy: 0.8649
Epoch 850/1000
250/250 [==============================] - 0s 293us/step - loss: 0.3306 - accuracy: 0.8645
Epoch 851/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3306 - accuracy: 0.8649
Epoch 852/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3307 - accuracy: 0.8636
Epoch 853/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3309 - accuracy: 0.8640
Epoch 854/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3306 - accuracy: 0.8648
Epoch 855/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3307 - accuracy: 0.8646
Epoch 856/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3307 - accuracy: 0.8656
Epoch 857/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3308 - accuracy: 0.8644
Epoch 858/1000
250/250 [==============================] - 0s 

250/250 [==============================] - 0s 297us/step - loss: 0.3303 - accuracy: 0.8639
Epoch 927/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3307 - accuracy: 0.8646
Epoch 928/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3306 - accuracy: 0.8652
Epoch 929/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3309 - accuracy: 0.8654
Epoch 930/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3304 - accuracy: 0.8645
Epoch 931/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3306 - accuracy: 0.8649
Epoch 932/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3308 - accuracy: 0.8651
Epoch 933/1000
250/250 [==============================] - 0s 292us/step - loss: 0.3307 - accuracy: 0.8652
Epoch 934/1000
250/250 [==============================] - 0s 291us/step - loss: 0.3310 - accuracy: 0.8649
Epoch 935/1000
250/250 [==============================] - 0s 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [18]:
y_pred = ann.predict(X_test)
y_pred

array([[0.25192118],
       [0.33648187],
       [0.2350232 ],
       ...,
       [0.14955378],
       [0.18565905],
       [0.20293221]], dtype=float32)

In [19]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1517   78]
 [ 202  203]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)